In [115]:
import numpy as np
import pandas as pd
import openpyxl 

from pandas.io.formats import excel
excel.ExcelFormatter.header_style = None
excel.ExcelFormatter.index_style = None

In [117]:
neg = pd.read_table("./input/output_report_neg.tsv")
pos = pd.read_table("./input/output_report_pos.tsv")
meta = pd.read_excel("./input/2023-07-27_extracted-yeast.xlsx",sheet_name="samples")


In [157]:
def convert_report(input, metain, charge, align_id_start):
    if charge == 1:
        seq = "Positive"
    else:
        seq = "Negative"

    keep = metain[metain["Sequence"]==seq]
    keep = np.array(keep["Sample_Type"].isin(["QC", "SAMPLE", "BLANK"]))

    tmp = input[input["Precursor Charge"] == charge]
    a = tmp.drop_duplicates(subset= ['Molecule Name', 'Precursor Mz'], keep = 'first')
    a = a[["Molecule Name", "Collisional Cross Section", "Precursor Mz"]]
    b = tmp["Replicate Name"].drop_duplicates(keep = 'first').values

    chrg = "POS" if charge == 1 else "NEG"

    alig_id = range(align_id_start, align_id_start + a.shape[0])

    d1 = pd.DataFrame({"Alignment ID": alig_id, 
                       "Molecule Name": a["Molecule Name"],
                       "Cross Collisional Section": a["Collisional Cross Section"],
                       "Average Mz": a["Precursor Mz"],
                       "Column": ["IM"] * a.shape[0],
                       "Ion Mode": [chrg] * a.shape[0]})
    d1.reset_index(drop = True, inplace= True)    

    areas = np.empty((0,a.shape[0]))
    for i in b:
        c = tmp[tmp["Replicate Name"] == i]["Area"].values
        areas = np.append(areas, [c], axis = 0)

    areas[np.isnan(areas)] = 0
    areas = np.transpose(areas)

    d2 = pd.DataFrame(areas, columns=b)
    print(len(keep))
    print(d2.shape)
    d2 = d2.loc[:,keep]
    d2 = pd.concat([pd.DataFrame({"Alignment ID": alig_id}), d2], axis=1)
    d2.reset_index(drop = True, inplace = True)

    return d1, d2

In [ ]:
def generate_meta(input):
    inj_order = range(1,input.shape[0]+1)
    qc = input["Sample_Type"]
    Group = input["Description"]
    replicate = input["Replicate_Number"]

    out = pd.DataFrame({
        "Injection_order": inj_order,
        "QC": qc,
        "Group": Group,
        "Replicate": replicate
    })

    out = out[out.QC.isin(["QC", "SAMPLE"])]

    outT = out.T
    return outT

In [158]:
posleft, posright = convert_report(pos, meta, 1, 0)
negleft, negright = convert_report(neg, meta, -1, posleft.shape[0])

369
(167, 369)
369
(200, 369)


In [153]:
seq = "Positive"

keep = meta[meta["Sequence"]==seq]
keep = keep["Sample_Type"].isin(["QC", "SAMPLE"])

In [154]:
len(keep)

369

In [159]:
outleft = pd.concat([posleft,negleft])

In [160]:
out = outleft.merge(posright, on="Alignment ID", how="outer")
out = out.merge(negright, on="Alignment ID", how="outer")

In [162]:
metaout = generate_meta(meta)

In [161]:
out

,Alignment ID,Molecule Name,Cross Collisional Section,Average Mz,Column,Ion Mode,Inj00002-Positive-A1_demultiplexed,Inj00003-Positive-A2_demultiplexed,Inj00004-Positive-A3_demultiplexed,Inj00005-Positive-A4_demultiplexed,...,Inj00360-Negative-P14_demultiplexed,Inj00361-Negative-P15_demultiplexed,Inj00362-Negative-P16_demultiplexed,Inj00363-Negative-P17_demultiplexed,Inj00364-Negative-P18_demultiplexed,Inj00365-Negative-P19_demultiplexed,Inj00366-Negative-P20_demultiplexed,Inj00367-Negative-P21_demultiplexed,Inj00368-Negative-P22_demultiplexed,Inj00369-Negative-P23_demultiplexed
0,0,(R)-Pantothenic acid-p1,129.8,220.117949,IM,POS,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2'-Deoxyadenosine-p1,152.3,252.109115,IM,POS,0.0,96.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2'-Deoxyadenosine-p2,158.2,252.109115,IM,POS,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,2-O-acetyl-1-O-hexadecyl-sn-glycero-3-phosphoc...,239.3,524.371066,IM,POS,11642.0,18283.0,19820.0,22313.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,"3',5'-cyclic AMP-p1",172.7,330.059796,IM,POS,2033.0,1087.0,371.0,1050.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
362,362,trans-4-Coumaric acid-p1,131.3,163.040068,IM,NEG,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
363,363,trans-4-hydroxy-L-proline-p1,126.7,130.050967,IM,NEG,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
364,364,trans-Aconitic acid-p1,131.4,173.009162,IM,NEG,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,1342.0,921.0,922.0,493.0,0.0,0.0
365,365,tryptophol-p1,133.0,160.076788,IM,NEG,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
test = out

In [22]:
test[['Average Mz', 'Cross Collisional Section']].value_counts() 

Average Mz  Cross Collisional Section
423.029868  178.1                        2
425.044420  182.0                        2
252.109115  152.3                        2
181.071762  131.9                        2
298.096836  171.0                        1
                                        ..
166.050967  129.2                        1
166.014582  128.0                        1
164.071703  140.6                        1
164.057784  136.1                        1
852.108328  252.2                        1
Name: count, Length: 363, dtype: int64

In [23]:
test2 = test.drop_duplicates(subset = ['Average Mz', 'Cross Collisional Section'], keep='first')

In [24]:
test2[['Average Mz', 'Cross Collisional Section']].value_counts() 

Average Mz  Cross Collisional Section
103.040068  120.5                        1
317.065579  171.9                        1
313.061151  167.5                        1
308.091082  167.9                        1
308.064213  164.5                        1
                                        ..
166.050967  129.2                        1
166.014582  128.0                        1
164.071703  140.6                        1
164.057784  136.1                        1
852.108328  252.2                        1
Name: count, Length: 363, dtype: int64

In [304]:
fn = r'./output/comb_data.xlsx'
with pd.ExcelWriter(fn) as writer:
    metaout.to_excel(writer, sheet_name='Sheet1', header=None, index=True, startcol=4, startrow=0)
    out.to_excel(writer, sheet_name='Sheet1', header=True, index=False, startcol=0,startrow=4)
# wb.save(filename=fn)

END

In [82]:
translist = pd.read_table('./input/ymdb_transition_list.csv', sep=',')

In [107]:
molecules = sorted(list(set(translist['Precursor Name'])))

In [105]:
molecules

In [108]:
print(molecules)

['(2)-Methylglutaric acid-p1', '(4-hydroxyphenyl)pyruvic acid-p1', '(R)-Mevalonic acid-p1', '(R)-Pantothenic acid-p1', "2'-Deoxyadenosine-p1", "2'-Deoxyadenosine-p2", "2'-Deoxycytidine-p1", "2'-Deoxyguanosine-p1", '2-Aminobenzoic acid-p1', '2-Dehydro-D-gluconate-p1', '2-Hydroxy-2-methylbutanedioic acid-p1', '2-O-acetyl-1-O-hexadecyl-sn-glycero-3-phosphocholine-p1', '2-hydroxybutyric acid-p1', '2-phospho-D-glyceric acid-p1', "3',5'-cyclic AMP-p1", "3',5'-cyclic AMP-p2", '3-(4-hydroxyphenyl)lactic acid-p1', '3-(indol-3-yl)pyruvic acid-p1', '3-Aminoisobutyrate-p1', '3-Dehydroshikimic acid-p1', '3-Hydroxyanthranilic acid-p1', '3-Hydroxybutyric acid-p1', '3-Methyladenine-p1', '4-Acetamidobutanoic acid-p1', '4-Aminobenzoic acid-p1', '4-Hydroxybenzoic acid-p1', '4-Hydroxyphenylacetic acid-p1', '4-Nitrophenol-p1', "5'-Deoxyadenosine-p1", "5'-Deoxyadenosine-p2", "5'-Methylthioadenosine-p1", '5-Amino-1-(5-phospho-D-ribosyl)imidazole-4-carboxamide-p1', '5-Aminolevulinic acid-p1', '5-Thymidylic ac

In [109]:
mol2 = [x[:-3] for x in molecules]
print(mol2)

['(2)-Methylglutaric acid', '(4-hydroxyphenyl)pyruvic acid', '(R)-Mevalonic acid', '(R)-Pantothenic acid', "2'-Deoxyadenosine", "2'-Deoxyadenosine", "2'-Deoxycytidine", "2'-Deoxyguanosine", '2-Aminobenzoic acid', '2-Dehydro-D-gluconate', '2-Hydroxy-2-methylbutanedioic acid', '2-O-acetyl-1-O-hexadecyl-sn-glycero-3-phosphocholine', '2-hydroxybutyric acid', '2-phospho-D-glyceric acid', "3',5'-cyclic AMP", "3',5'-cyclic AMP", '3-(4-hydroxyphenyl)lactic acid', '3-(indol-3-yl)pyruvic acid', '3-Aminoisobutyrate', '3-Dehydroshikimic acid', '3-Hydroxyanthranilic acid', '3-Hydroxybutyric acid', '3-Methyladenine', '4-Acetamidobutanoic acid', '4-Aminobenzoic acid', '4-Hydroxybenzoic acid', '4-Hydroxyphenylacetic acid', '4-Nitrophenol', "5'-Deoxyadenosine", "5'-Deoxyadenosine", "5'-Methylthioadenosine", '5-Amino-1-(5-phospho-D-ribosyl)imidazole-4-carboxamide', '5-Aminolevulinic acid', '5-Thymidylic acid', '6-Phosphogluconic acid', '7-Methylguanine', 'ADP', 'Adenine', 'Adenosine diphosphate ribose',

In [114]:
df = pd.DataFrame({"Names": mol2})

df.to_clipboard(header=False, index=False)